# Attribute table

Primary input for the Panel distribution algorithm is an attribute table
This script develops the attribute table which consists of rooftop segment ID, geometry of rooftop segment, slope(β) and aspect(α) of the rooftop segment

In [ ]:
import numpy as np;
import pandas as pd
from PIL import Image
import statistics
import geopandas as gpd
import math
from osgeo import gdal, osr
import time
from shapely.geos import TopologicalError
from shapely.geometry import LineString, mapping
from itertools import combinations
import re

In [ ]:
    """
    Input parameters:
    -----------------
    
    The primary inputs for this script are raster and vector datasets.
    
    building_id_raster: Vector shapefile which contains LOD2 roof segments is converted into raster dataset with pixels 
                        representing the segment_id of the building.
                        This file is used to extract pixels that belong to a particular segments.
    Reclassified_slope: Slope of a segment is calculated from DSM by using Geospatial Data Abstraction Library 
                        (GDAL) Spatial Analyst toolbox in QGIS.
                        After generating the slope raster, slopes are reclassified.In this reclassification, 
                        slopes higher than 60° are considered walls and hence eliminated by allocating slopes 
                        ranging in between 60° and 90° to -1
    Aspect_clipped    : The aspect tool of the GDAL Spatial Analyst Toolbox in QGIS was used to obtain aspect values from DSM
    shapefile         : Vector shapefile which contains LOD2 roof segments with an attribute column 'segment_id'(@row_number)
    flat_slope        : Optimized slope - must be oprimized according to the location of the study.
        
        
    """
# Input raster and vector files

building_id_raster = gdal.Open('Building_id_raster.tif')
Reclassified_slope = gdal.Open('Reclassified_slope.tif') 
Aspect_clipped = gdal.Open('Aspect.tif')
shapefile = gpd.read_file("vector_area.shp")
flat_slope = 40 

In [ ]:
# Converting raster files into python dataframes.

building = building_id_raster.GetRasterBand(1).ReadAsArray()
slope = Reclassified_slope.GetRasterBand(1).ReadAsArray()
aspect = Aspect_clipped.GetRasterBand(1).ReadAsArray()

# Getting unique building/segment list
building_list = np.unique(building)
building_list = building_list[building_list!=0]
geo_transform = building_id_raster.GetGeoTransform ()

In [ ]:
for i in building_list:
    
    geo = shapefile.loc[shapefile['segment_id'] == i, 'geometry'].iloc[0]
    cen = geo.centroid
    
    result = np.where(building == i)
    pixels = list(zip(*result))
    slope_raster = []
    aspect_raster = []
    unwanted_pixel = [-1]
    
    pixel_x = int((cen.x - geo_transform[0]) // geo_transform[1])
    pixel_y = int((cen.y - geo_transform[3]) // (geo_transform[5]))
    
    for pixel in pixels:   # appending slopes and aspect that belonging to same roof segment.
        slope_raster.append(slope[pixel])
        aspect_raster.append(aspect[pixel])

    if slope_raster.count(slope_raster[0]) == len(slope_raster):
      pass
    
    else:
        while unwanted_pixel in slope_raster: slope_raster.remove(unwanted_pixel)
            # Slope raster vary across a rooftop segment due to some error in input (LiDAR) data
            # Hence, to minimize the error and get a single slope value for a rooftop segment, 
            # the median filter was used over all the slope pixels belonging to a particular rooftop segment.
        slope_value = statistics.median(slope_raster) # 
    
        if slope_value < 10: # Slopes below 10° are assumed to be flat surfaces
            shapefile.loc[shapefile.segment_id ==i, 'act_area'] = shapefile.loc[shapefile.segment_id ==i, 'area']
            shapefile.loc[shapefile.segment_id ==i, 'Slope'] = flat_slope
            shapefile.loc[shapefile.segment_id ==i, 'Aspect'] = 180

            
        else:
            shapefile.loc[shapefile.segment_id ==i, 'act_area'] = shapefile.loc[shapefile.segment_id ==i, 'area'] / math.cos(math.radians(slope_value))
            shapefile.loc[shapefile.segment_id ==i, 'Slope'] = slope_value
            shapefile.loc[shapefile.segment_id ==i, 'Aspect'] = aspect[pixel_y, pixel_x]

In [ ]:
""""
    Like slope raster, aspect raster data also contains noise and to get a single aspect value 
    for a tilted rooftop segment, the line string method has been used.
    A line string is a shape that has a dimension of 1. It is a simple line that does not intersect itself. 
    Every polygon is made up of several line strings, which act as a boundary of that polygon. 
    Since the rooftop segment is also a polygon, each rooftop segment contains line strings, and the slope of 
    the largest line string is taken as the aspect of the segment because the PV panels on the rooftops are 
    installed along the edges.
""""

for i in range(shapefile.shape[0]):
    longest_lines = []      
    g = shapefile.loc[i, 'geometry']        
    if (g.geom_type == 'Polygon') and (shapefile.loc[i, 'Slope'] != flat_slope):
        x,y = g.exterior.coords.xy
        coords = np.dstack((x,y)).tolist()
        vertices = []
        for x in coords:
            for z in x:
                vertices.append(z)
        
        if len(vertices)>2:
            lines = []
            for j in range(len(vertices)-1):
                if j == (len(vertices) - 2) :
                    line = LineString([(float(vertices[0][0]),float(vertices[0][1])), (float(vertices[j][0]),float(vertices[j][1]))])
                else:
                    line = LineString([(float(vertices[j][0]),float(vertices[j][1])), (float(vertices[j+1][0]),float(vertices[j+1][1]))])
                lines.append(line)
            longest_lines.append(max(lines, key=lambda x: x.length)) #Find longest line
            x,y = longest_lines[0].coords.xy
            pd.DataFrame({'LAT':x,'LON':y})
            shapefile.loc[i, 'Seg_Aspect'] = np.degrees(math.atan2((y[1] - y[0]),(x[1] - x[0])))
    else:
        shapefile.loc[i, 'Seg_Aspect'] = shapefile.loc[i, 'Aspect']

In [ ]:
shapefile.dropna(inplace=True)
print("--- %s seconds ---" % (time.time() - start_time)) 
outfp = r"segments.shp"
shapefile.to_file(outfp)